**Energy Calculations**

Here we start with the simplest type of calculation - a "single point" energy calculation, in this case for water.

For all calculations, we need to start by import some Python objects to create an H2O object, and run calculations (which are performed using an `EMT` forcefield calculator, based on Newtonian principles)

In [1]:
from ase import Atoms
from ase.calculators.emt import EMT

Now we need to define our water molecule. We define the composition of the molecule, the positions of the atoms in Cartesian (3D) space, and the calculator for working out an energy:

In [2]:
water = Atoms('H2O',
               positions=[(1, 0, 0),
                          (0, 1, 0),
                          (0, 0, 0)],
               calculator=EMT())

With this, we can calculate the energy with a command like:

In [3]:
print(water.get_potential_energy())

2.3137464701454697


Great! But how is this useful? Well, the important thing is for us to work with relative energies. We can think about potential energy landscapes by for instance varying the bond lengths and checking the energy thus:

In [4]:
from ase.io.trajectory import Trajectory
traj = Trajectory('h2o.traj', 'w')

d = 1.0
for i in range(10):
    water = Atoms('H2O',
                  positions=[(i*d, 0, 0),
                             (0, i*d, 0),
                             (0, 0, 0)],
                  calculator=EMT())
    print("Bond length: ", i*d, "; Energy: ", water.get_potential_energy())
    traj.write(water)

Bond length:  0.0 ; Energy:  756.5078742552881
Bond length:  1.0 ; Energy:  2.3137464701454697
Bond length:  2.0 ; Energy:  8.238084038523857
Bond length:  3.0 ; Energy:  10.669544119641944
Bond length:  4.0 ; Energy:  10.984679654560367
Bond length:  5.0 ; Energy:  11.017004375870611
Bond length:  6.0 ; Energy:  11.02
Bond length:  7.0 ; Energy:  11.02
Bond length:  8.0 ; Energy:  11.02
Bond length:  9.0 ; Energy:  11.02


/Users/andrewlogsdail/Library/Python/3.8/lib/python/site-packages/ase/calculators/emt.py:218: RuntimeWarning: invalid value encountered in true_divide
  f = ((y1 * p2['kappa'] + y2 * p1['kappa']) / beta +
/Users/andrewlogsdail/Library/Python/3.8/lib/python/site-packages/ase/calculators/emt.py:235: RuntimeWarning: invalid value encountered in true_divide
  f = ((y1 * p2['eta2'] + y2 * p1['eta2']) +


Note how this looks the same as our previous definition for H2O, but the positions are now dependent on the variables `i*d` - i.e. they are not constant. We also see that the energy is very high at low distance, at a minimum close to 1.0 Angstrom, and then tends to a constant at longer distances. We can visualise the water models thus:

In [5]:
from ase.visualize import view
traj = Trajectory("h2o.traj")
view(traj, viewer='ngl')

/Users/andrewlogsdail/Library/Python/3.8/lib/python/site-packages/ase/io/jsonio.py:122: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  a = np.array(obj)


/Users/andrewlogsdail/Library/Python/3.8/lib/python/site-packages/ase/io/jsonio.py:122: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  a = np.array(obj)


_Note_: Might not work on the remote binder facility; will work on a local computer if you just remove the `x3d` or `ngl` options.

**Geometry Optimisation**

In reality, we don't normally want to run linear scans as the dimensionality of the problem is much greater than one or two bonds (think of a many atom molecule!), so we use an automated process of geometry optimisations instead. Again we define our water molecule, and then perform the optimisation:

In [6]:
from ase.optimize import BFGS

# Create a linear water molecule
d = 1.0
water = Atoms('H2O',
              positions=[(d, 0, 0),
                         (-d, 0, 0),
                         (0, 0, 0)],
              calculator=EMT())

# Run the calculation
dyn = BFGS(water, trajectory='h2o.traj')
dyn.run(fmax=0.05)

      Step     Time          Energy         fmax
BFGS:    0 20:50:18        2.266718        4.2914
BFGS:    1 20:50:18        2.007585        2.2564
BFGS:    2 20:50:18        1.895482        0.6048
BFGS:    3 20:50:18        1.885496        0.0838
BFGS:    4 20:50:18        1.885290        0.0039


True

The ouput here gives a step number, the time, the energy of the model and the maximum force (`fmax`), which in this example we want to be below 0.05 eV / Angstrom to find a stable structure. See how the force falls to a minimum value? Try playing with the starting geometry and seeing if you can get any other results. You can view the output trajcetory and final structure using the command below again (what do you notice unusual about the final structure?):

In [7]:
traj = Trajectory('h2o.traj')
view(traj, viewer='ngl')